In [ ]:
#import the desired packages

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import time
import csv

In [ ]:
#launch url
url = "https://www.wellsfargojobs.com/search-jobs"

# create a new Chrome session
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

#Click on the Accept button on the cookie consent banner
cookie_button = driver.find_element_by_id('gdpr-button')
cookie_button.click()

In [ ]:
#Create an empty list for the URLs
url_list = []

#Run the loop for number of pages in the careers section
for page in range(1,63):
    soup_level1=BeautifulSoup(driver.page_source, 'lxml') #Find the current page source
    
    #iterate through the list of jobs on each of page
    for link in soup_level1.find_all( 'a', href=re.compile("/job/*")):
        url_list.append(link.get('href'))
    
    #Find and click on the next button after each list is iterated
    next_button = driver.find_element_by_xpath("//a[@class ='next']")
    next_button.click()
    time.sleep(20)
    
driver.quit() # close the open browser session

In [ ]:
with open('wellsfargo_jobs.csv', mode='w', encoding="utf-8") as csv_file:  #Create a new csv file
    fieldnames = ['job_url', 'title', 'job_id', 'location', 'description'] #Define the fields 
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    
    #Iterate through each of the job urls
    for job_url in url_list:
        company_url = "https://www.wellsfargojobs.com"
        visit_url = company_url + job_url
        driver = webdriver.Chrome()
        driver.get(visit_url)
         
        soup = BeautifulSoup(driver.page_source, 'lxml') #Find the current page source
        
        #Find the desired fields
        title = soup.find('h1', itemprop = True)
        if str(title)=="None":
            title = soup.find('h1', {'class':'ajd_header__job-title'})

        job_id = soup.find('span', {'class':'job-id job-info'})
        if str(job_id)=='None':
            job_id = soup.find('p', {'class':'ajd_header__jobid'})#
    
        location = soup.find_all('span', {'class':'job-info'})
        if location==[]:
            location.append('fillertext')
        location.append(soup.find('p', {'class':'ajd_header__location'}))
        
        desc = soup.find('div', {'class':'ats-description' or "ats-description ajd_job-details__ats-description"})
        

        #Write each row to the csv file
        writer.writerow({'job_url':visit_url, 'title':title.get_text(), 'job_id':job_id.get_text(), 'location':location[1].text, 'description':desc.get_text()})
        print(visit_url)
        driver.quit()   #Close the open browser session